In [35]:
import json
import pandas as pd
import numpy as np
import os
import sys
import tensorflow as tf
import tensorflow_hub as hub
from time import time
import io
import re
from csv import reader
import matplotlib.pyplot as plt

import matplotlib.pyplot as plt
from matplotlib import colors
from matplotlib.ticker import PercentFormatter
from tqdm import tqdm_notebook

from tensorflow.keras import backend as K
from tensorflow.keras import layers
from tensorflow.python.keras.layers import Lambda
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.backend import sparse_categorical_crossentropy
from tensorflow.keras.layers import Dense, TimeDistributed

from sklearn.model_selection import train_test_split

import datetime


In [3]:
local_bert_path =   '/Users/fohanlin/PycharmProjects/w266NLP/bert-master/' # change as needed 

now = datetime.now() # current date and time

# make sure that the paths are accessible within the notebook
sys.path.insert(0,local_bert_path)

In [4]:
sys.path

['/Users/fohanlin/PycharmProjects/w266NLP/bert-master/',
 '/Users/fohanlin/PycharmProjects/w266NLP',
 '/Library/Frameworks/Python.framework/Versions/3.6/lib/python36.zip',
 '/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6',
 '/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/lib-dynload',
 '',
 '/Users/fohanlin/PycharmProjects/w266NLP/venv/lib/python3.6/site-packages',
 '/Users/fohanlin/PycharmProjects/w266NLP/venv/lib/python3.6/site-packages/setuptools-39.1.0-py3.6.egg',
 '/Users/fohanlin/PycharmProjects/w266NLP/venv/lib/python3.6/site-packages/IPython/extensions',
 '/Users/fohanlin/.ipython']

In [5]:
import optimization
import run_classifier
import tokenization
import run_classifier_with_tfhub

## Data Preprocess

In [8]:
# Read in the statements
statements = pd.read_csv("fomc_statement.csv",parse_dates=["Date",])
yields = pd.read_csv("DailyTreasuryYield.csv",parse_dates=["Date",])

# Correct one FOMC website mistake
statements.loc[statements["Date"] == datetime.date(2007, 6, 18), "Date"] = datetime.date(2007, 6, 28)
statements.sort_values(by='Date', inplace=True)
# Change a weekend to the latest business date
statements.loc[statements["Date"] == datetime.date(2010, 5, 9), "Date"] = datetime.date(2010, 5, 7)

result = pd.merge(yields, statements, on='Date', how='left')
result['10YR_3MO'] = result['10 YR']-result['3 MO']
result['d3MO'] = result['3 MO'].diff()
result['d_structure'] = result['10YR_3MO'].diff()

In [68]:
# generate rates movement for each release date
myDates = result[result['Statement'].notnull()]['Date']
window = 1 # business days
for i in range(len(myDates)):
    start_index = myDates.index[i]
    if i == len(myDates) - 1:
        policy_interim = np.inf
    else: 
        policy_interim = myDates.index[i+1] - myDates.index[i]
    sub_result = result.iloc[start_index:min(start_index+window,start_index+policy_interim)]
    result.loc[result["Date"] == myDates.iloc[i], "3MO_move"] = sub_result['d3MO'].sum()
    #result.loc[result["Date"] == myDates.iloc[i], "3MO_std"] = sub_result['d3MO'].std()
    result.loc[result["Date"] == myDates.iloc[i], "structure_move"] = sub_result['d_structure'].sum()
    #result.loc[result["Date"] == myDates.iloc[i], "structure_std"] = sub_result['d_structure'].std()

In [69]:
out = result[result['Statement'].notnull()]

out['3MO_label'] = out['3MO_move'] > 0
out['structure_label'] = out['structure_move'] > 0

/Users/fohanlin/PycharmProjects/w266NLP/venv/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/fohanlin/PycharmProjects/w266NLP/venv/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [70]:
out.tail(20)

,Date,1 MO,3 MO,6 MO,1 YR,2 YR,3 YR,5 YR,7 YR,10 YR,...,Statement,10YR_3MO,d3MO,d_structure,3MO_move,3MO_std,structure_move,structure_std,3MO_label,structure_label
6839,2017-05-03,0.73,0.85,1.00,1.10,1.30,1.50,1.86,2.14,2.33,...,Information received since the Federal Open Ma...,1.48,0.03,0.01,0.03,0.015275,0.01,0.032146,True,True
6868,2017-06-14,0.90,1.01,1.12,1.20,1.35,1.48,1.74,1.96,2.15,...,Information received since the Federal Open Ma...,1.14,0.01,-0.07,0.01,0.000000,-0.07,0.037859,True,False
6897,2017-07-26,1.02,1.13,1.14,1.23,1.36,1.50,1.83,2.09,2.29,...,Information received since the Federal Open Ma...,1.16,-0.05,0.01,-0.05,0.015275,0.01,0.023094,False,True
6936,2017-09-20,0.98,1.04,1.20,1.32,1.45,1.60,1.89,2.12,2.28,...,Information received since the Federal Open Ma...,1.24,0.00,0.04,0.00,0.005774,0.04,0.026458,False,True
6965,2017-11-01,1.06,1.18,1.30,1.46,1.61,1.74,2.01,2.22,2.37,...,Information received since the Federal Open Ma...,1.19,0.03,-0.04,0.03,0.020000,-0.04,0.015275,True,False
6994,2017-12-13,1.22,1.30,1.47,1.68,1.79,1.90,2.12,2.26,2.36,...,Information received since the Federal Open Ma...,1.06,-0.04,0.00,-0.04,0.030000,0.00,0.020817,False,False
7026,2018-01-31,1.43,1.46,1.66,1.90,2.14,2.29,2.52,2.66,2.72,...,Information received since the Federal Open Ma...,1.26,0.02,-0.03,0.02,0.011547,-0.03,0.047258,True,False
7060,2018-03-21,1.71,1.74,1.95,2.06,2.31,2.46,2.69,2.82,2.89,...,Information received since the Federal Open Ma...,1.15,-0.07,0.07,-0.07,0.045092,0.07,0.060828,False,True
7089,2018-05-02,1.69,1.84,2.03,2.24,2.49,2.64,2.80,2.92,2.97,...,Information received since the Federal Open Ma...,1.13,-0.01,0.01,-0.01,0.005774,0.01,0.023094,False,True
7118,2018-06-13,1.82,1.94,2.09,2.35,2.59,2.71,2.85,2.95,2.98,...,Information received since the Federal Open Ma...,1.04,0.02,0.00,0.02,0.011547,0.00,0.020817,True,False


In [71]:
# Define maximal length of input 'sentences' (post tokenization).
max_length = 500
max_seq_length = 500
train_text = out['Statement'].tolist()[:164]
train_text = [' '.join(t.split()[0:max_seq_length]) for t in train_text]
train_text = np.array(train_text, dtype=object)[:, np.newaxis]
train_label = out['3MO_label'].tolist()[:164]

test_text = out['Statement'].tolist()[-20:]
test_text = [' '.join(t.split()[0:max_seq_length]) for t in test_text]
test_text = np.array(test_text, dtype=object)[:, np.newaxis]
test_label = out['3MO_label'].tolist()[-20:]


In [72]:
# Tensorflow hub path to BERT module of choice
bert_url = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"

In [73]:
class PaddingInputExample(object):
    """Fake example so the num input examples is a multiple of the batch size.
  When running eval/predict on the TPU, we need to pad the number of examples
  to be a multiple of the batch size, because the TPU requires a fixed batch
  size. The alternative is to drop the last batch, which is bad because it means
  the entire output data won't be generated.
  We use this class instead of `None` because treating `None` as padding
  battches could cause silent errors.
  """

class InputExample(object):
    """A single training/test example for simple sequence classification."""

    def __init__(self, guid, text_a, text_b=None, label=None):
        """Constructs a InputExample.
    Args:
      guid: Unique id for the example.
      text_a: string. The untokenized text of the first sequence. For single
        sequence tasks, only this sequence must be specified.
      text_b: (Optional) string. The untokenized text of the second sequence.
        Only must be specified for sequence pair tasks.
      label: (Optional) string. The label of the example. This should be
        specified for train and dev examples, but not for test examples.
    """
        self.guid = guid
        self.text_a = text_a
        self.text_b = text_b
        self.label = label

def create_tokenizer_from_hub_module():
    """Get the vocab file and casing info from the Hub module."""
    with tf.Graph().as_default():
        bert_module = hub.Module(bert_url)
        tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
        with tf.Session() as sess:
            vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                            tokenization_info["do_lower_case"]])
      
    return tokenization.FullTokenizer(
      vocab_file=vocab_file, do_lower_case=do_lower_case)

def convert_single_example(tokenizer, example, max_seq_length=256):
    """Converts a single `InputExample` into a single `InputFeatures`."""

    if isinstance(example, PaddingInputExample):
        input_ids = [0] * max_seq_length
        input_mask = [0] * max_seq_length
        segment_ids = [0] * max_seq_length
        label = 0
        return input_ids, input_mask, segment_ids, label

    tokens_a = tokenizer.tokenize(example.text_a)
    if len(tokens_a) > max_seq_length - 2:
        tokens_a = tokens_a[0 : (max_seq_length - 2)]

    tokens = []
    segment_ids = []
    tokens.append("[CLS]")
    segment_ids.append(0)
    for token in tokens_a:
        tokens.append(token)
        segment_ids.append(0)
    tokens.append("[SEP]")
    segment_ids.append(0)

    input_ids = tokenizer.convert_tokens_to_ids(tokens)

    # The mask has 1 for real tokens and 0 for padding tokens. Only real
    # tokens are attended to.
    input_mask = [1] * len(input_ids)

    # Zero-pad up to the sequence length.
    while len(input_ids) < max_seq_length:
        input_ids.append(0)
        input_mask.append(0)
        segment_ids.append(0)

    assert len(input_ids) == max_seq_length
    assert len(input_mask) == max_seq_length
    assert len(segment_ids) == max_seq_length

    return input_ids, input_mask, segment_ids, example.label

def convert_examples_to_features(tokenizer, examples, max_seq_length=256):
    """Convert a set of `InputExample`s to a list of `InputFeatures`."""

    input_ids, input_masks, segment_ids, labels = [], [], [], []
    for example in tqdm_notebook(examples, desc="Converting examples to features"):
        input_id, input_mask, segment_id, label = convert_single_example(
            tokenizer, example, max_seq_length
        )
        input_ids.append(input_id)
        input_masks.append(input_mask)
        segment_ids.append(segment_id)
        labels.append(label)
    return (
        np.array(input_ids),
        np.array(input_masks),
        np.array(segment_ids),
        np.array(labels).reshape(-1, 1),
    )

def convert_text_to_examples(texts, labels):
    """Create InputExamples"""
    InputExamples = []
    for text, label in zip(texts, labels):
        InputExamples.append(
            InputExample(guid=None, text_a=" ".join(text), text_b=None, label=label)
        )
    return InputExamples

In [14]:
# Instantiate tokenizer
tokenizer = create_tokenizer_from_hub_module()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [74]:
# Convert data to InputExample format
train_examples = convert_text_to_examples(train_text, train_label)
test_examples = convert_text_to_examples(test_text, test_label)

# Convert to features
(train_input_ids, train_input_masks, train_segment_ids, train_labels 
) = convert_examples_to_features(tokenizer, train_examples, max_seq_length=max_seq_length)
(test_input_ids, test_input_masks, test_segment_ids, test_labels
) = convert_examples_to_features(tokenizer, test_examples, max_seq_length=max_seq_length)

/Users/fohanlin/PycharmProjects/w266NLP/venv/lib/python3.6/site-packages/ipykernel_launcher.py:88: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


In [75]:
len(test_input_ids[0])

500

In [76]:
adam_customized = tf.keras.optimizers.Adam(lr=0.001, beta_1=0.91, beta_2=0.999, epsilon=None, decay=0.1, amsgrad=False)

In [77]:
class BertLayer(tf.keras.layers.Layer):
    """
    Create BERT layer, following https://towardsdatascience.com/bert-in-keras-with-tensorflow-hub-76bcbc9417b
    init:  initialize layer. Specify various parameters regarding output types and dimensions. Very important is
           to set the number of trainable layers.
    build: build the layer based on parameters
    call:  call the BERT layer within a model
    """
    
    def __init__(
        self,
        n_fine_tune_layers=10,
        pooling="first",
        bert_url="https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1",
        **kwargs,
    ):
        self.n_fine_tune_layers = n_fine_tune_layers
        self.trainable = True
        self.output_size = 768
        self.pooling = pooling
        self.bert_url = bert_url
        if self.pooling not in ["first", "mean"]:
            raise NameError(
                f"Undefined pooling type (must be either first or mean, but is {self.pooling}"
            )

        super(BertLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.bert = hub.Module(
            self.bert_url, trainable=self.trainable, name=f"{self.name}_module"
        )

        # Remove unused layers
        trainable_vars = self.bert.variables
        if self.pooling == "first":
            trainable_vars = [var for var in trainable_vars if not "/cls/" in var.name]
            trainable_layers = ["pooler/dense"]

        elif self.pooling == "mean":
            trainable_vars = [
                var
                for var in trainable_vars
                if not "/cls/" in var.name and not "/pooler/" in var.name
            ]
            trainable_layers = []
        else:
            raise NameError(
                f"Undefined pooling type (must be either first or mean, but is {self.pooling}"
            )


        # Select how many layers to fine tune
        for i in range(self.n_fine_tune_layers):
            trainable_layers.append(f"encoder/layer_{str(11 - i)}")

        # Update trainable vars to contain only the specified layers
        trainable_vars = [
            var
            for var in trainable_vars
            if any([l in var.name for l in trainable_layers])
        ]

        # Add to trainable weights
        for var in trainable_vars:
            self._trainable_weights.append(var)

        for var in self.bert.variables:
            if var not in self._trainable_weights:
                self._non_trainable_weights.append(var)

        super(BertLayer, self).build(input_shape)

    def call(self, inputs):
        inputs = [K.cast(x, dtype="int32") for x in inputs]
        input_ids, input_mask, segment_ids = inputs
        bert_inputs = dict(
            input_ids=input_ids, input_mask=input_mask, segment_ids=segment_ids
        )
        if self.pooling == "first":
            pooled = self.bert(inputs=bert_inputs, signature="tokens", as_dict=True)[
                "pooled_output"
            ]
        elif self.pooling == "mean":
            result = self.bert(inputs=bert_inputs, signature="tokens", as_dict=True)[
                "sequence_output"
            ]

            mul_mask = lambda x, m: x * tf.expand_dims(m, axis=-1)
            masked_reduce_mean = lambda x, m: tf.reduce_sum(mul_mask(x, m), axis=1) / (
                    tf.reduce_sum(m, axis=1, keepdims=True) + 1e-10)
            input_mask = tf.cast(input_mask, tf.float32)
            pooled = masked_reduce_mean(result, input_mask)
        else:
            raise NameError(f"Undefined pooling type (must be either first or mean, but is {self.pooling}")

        return pooled

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_size)

In [78]:
def variable_summaries(var):
    """Attach a lot of summaries to a Tensor (for TensorBoard visualization)."""
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean', mean)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.summary.scalar('stddev', stddev)
        tf.summary.scalar('max', tf.reduce_max(var))
        tf.summary.scalar('min', tf.reduce_min(var))
        tf.summary.histogram('histogram', var)

In [79]:
def text_model(max_input_length, train_layers, optimizer):
    """
    Implementation of NER model
    
    variables:
        max_input_length: number of tokens (max_length + 1)
        train_layers: number of layers to be retrained
        optimizer: optimizer to be used
    
    returns: model
    """
    
    in_id = tf.keras.layers.Input(shape=(max_length,), name="input_ids")
    in_mask = tf.keras.layers.Input(shape=(max_length,), name="input_masks")
    in_segment = tf.keras.layers.Input(shape=(max_length,), name="segment_ids")
    bert_inputs = [in_id, in_mask, in_segment]
    
    bert_sequence = BertLayer(n_fine_tune_layers=train_layers, pooling="first")(bert_inputs)
    
    print(bert_sequence)
    
    dense = tf.keras.layers.Dense(256, activation='relu', name='dense')(bert_sequence)
    
    dense = tf.keras.layers.Dropout(rate=0.1)(dense)
    
    pred = tf.keras.layers.Dense(1, activation='sigmoid', name='textclass')(dense)
     
    print('pred: ', pred)
    
    ## Prepare for multipe loss functions, although not used here
    
    model = tf.keras.models.Model(inputs=bert_inputs, outputs=pred)

    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
    
    model.summary()
    
    return model


In [80]:
def initialize_vars(sess):
    sess.run(tf.local_variables_initializer())
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    K.set_session(sess)

In [81]:
#Start session

sess = tf.Session()

model = text_model(max_length + 1, train_layers=0, optimizer = adam_customized)

# Instantiate variables
initialize_vars(sess)

tensorboard = TensorBoard(log_dir="logs/{}".format(time()))

model.fit(
    [train_input_ids, train_input_masks, train_segment_ids], 
    train_labels,
    validation_data=([test_input_ids, test_input_masks, test_segment_ids], test_labels),
    epochs=5,
    batch_size=16#,
    #callbacks=[tensorboard]
)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Tensor("bert_layer_3/bert_layer_3_module_apply_tokens/bert/pooler/dense/Tanh:0", shape=(?, 768), dtype=float32)
pred:  Tensor("textclass_3/Sigmoid:0", shape=(?, 1), dtype=float32)
Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 500)]        0                                            
__________________________________________________________________________________________________
input_masks (InputLayer)        [(None, 500)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 500)]        0                                            
__________________________________________________________________________________________________
bert_layer_

In [61]:
print(sess._closed)

False


In [83]:
test_labels

array([[ True],
       [ True],
       [False],
       [False],
       [ True],
       [False],
       [ True],
       [False],
       [False],
       [ True],
       [False],
       [False],
       [False],
       [ True],
       [False],
       [ True],
       [False],
       [False],
       [False],
       [False]])

In [65]:
model.save('BertModel.h6')
pre_save_preds = model.predict([test_input_ids[:20], 
                               test_input_masks[:20], 
                               test_segment_ids[:20]]
                                ) # predictions before we clear and reload model

# Clear and load model
model = None
model = text_model(max_length + 1, train_layers=0, optimizer = adam_customized)
initialize_vars(sess)
model.load_weights('BertModel.h6')

post_save_preds = model.predict([test_input_ids[:20],
                                test_input_masks[:20], 
                                test_segment_ids[:20]]
                              ) # predictions after we clear and reload model
#all(pre_save_preds == post_save_preds) # Are they the same?

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Tensor("bert_layer_2/bert_layer_2_module_apply_tokens/bert/pooler/dense/Tanh:0", shape=(?, 768), dtype=float32)
pred:  Tensor("textclass_2/Sigmoid:0", shape=(?, 1), dtype=float32)
Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 500)]        0                                            
__________________________________________________________________________________________________
input_masks (InputLayer)        [(None, 500)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 500)]        0                                            
__________________________________________________________________________________________________
bert_layer_

In [66]:
pre_save_preds

array([[0.3230358 ],
       [0.32791266],
       [0.32135642],
       [0.34893155],
       [0.3405346 ],
       [0.33953637],
       [0.39936844],
       [0.38097608],
       [0.39637637],
       [0.41483054],
       [0.41104797],
       [0.41575128],
       [0.4108116 ],
       [0.42024577],
       [0.41359425],
       [0.4108062 ],
       [0.42806455],
       [0.41383368],
       [0.39492103],
       [0.39688128]], dtype=float32)

In [67]:
post_save_preds

array([[0.3230358 ],
       [0.32791266],
       [0.32135642],
       [0.34893155],
       [0.3405346 ],
       [0.33953637],
       [0.39936844],
       [0.38097608],
       [0.39637637],
       [0.41483054],
       [0.41104797],
       [0.41575128],
       [0.4108116 ],
       [0.42024577],
       [0.41359425],
       [0.4108062 ],
       [0.42806455],
       [0.41383368],
       [0.39492103],
       [0.39688128]], dtype=float32)

In [ ]:
sess.close()